# Oil Density Estimation

## A Description of the Methods Used (Including Some Examples)

### Background:

The density of a substance is a fairly important piece of information to know in regards to oil spill response.  If an oil is released into a body of water, its relative density to that body of water will indicate whether the oil will float on top of the water surface, or whether it will gradually sink below the surface.

There are two general properties in an oil record that indicate its initial density.  Those are the oils **API** gravity, and its **measured density**.

An oil's API gravity is a measure of how heavy or light it is compared to water, as defined by the
**A**merican **P**etroleum **I**nstitute. As such it is essentially a different unit for the density of the oil at standard temperature (and pressure). If there is a measured density (at 15C) and an API in a given record, the values should match, or there is an error in the record.


**API** gravity is defined by the following formula:

$$
\begin{align}
\rho_{0_{oil}} &= \text{The initial oil density at } 15^\circ C \text{ and measured in } kg / m^3 \\
1000 &= \text{The initial density of fresh water measured in } kg / m^3 \\
API &= \frac{141.5 \cdot 1000}{\rho_{0_{oil}} } - 131.5 \\
\end{align}
$$

Conversely, the initial density of the oil can be calculated, if necessary, from the API value

$$
\begin{align}
\rho_{0_{oil}} &= \frac{141.5}{131.5 + API} \cdot 1000 \\
\end{align}
$$

### Some Examples

Ok, let's apply this to some examples using records in our oil library.

First we will import some things from our oil library that we will need.

In [10]:
from oil_library import get_oil, get_oil_props, _get_db_session
from oil_library.models import (Base, ImportedRecord, Oil,
                                Density, Toxicity, Category)
from oil_library.oil_props import OilProps

session = _get_db_session()

Now we can query for an oil that we are interested in.

#### Example 1:  Estimating Density 

For the first example we will use a reasonably heavy oil for which we have no measured density values. (i.e. only API)

In [11]:
oil_obj = (session.query(Oil)
           .filter(Oil.name == 'BOSCAN, OIL & GAS')
           .one())
print oil_obj

<Oil("BOSCAN, OIL & GAS")>


In [12]:
print oil_obj.imported.densities
print oil_obj.imported.api

[]
10.1


First, a little bit of background.  The records in the PyGnome oil library have been imported from a master copy of the NOAA oil library data, and the original values are stored in the 'imported' attribute of the oil object.<br>
Here we can see that the original imported record contains no measured densities, but has a valid API value.<br>
And as discussed above, the density at $ 15^\circ C $ in $kg/m^3$ can be converted from the API as follows:

In [13]:
density_at_15c = 141.5 / (131.5 + 10.1) * 1000
print density_at_15c

999.293785311


So for this oil, we give it a single density at $ 15^\circ C $ since the imported record did not supply any.

In [28]:
print oil_obj.densities
print oil_obj.api

[<Density(660.0 kg/m^3 at 288.15K)>]
82.8939393939


At this point it is worth mentioning that internally, we use temperatures that are measured in Kelvin.<br>
So $ 15^\circ C $ converted into Kelvin would be:

$$
\begin{align}
273.15 + 15^\circ C &= 288.15^\circ K
\end{align}
$$

#### Example 2:  Estimating API

Ok, now we will examine an oil that has no API, but does in fact have some measured densities.

In [15]:
oil_obj = (session.query(Oil)
           .filter(Oil.name == 'COOPER BASIN LIGHT NAPHTHA, SANTOS LTD')
           .one())
print oil_obj
print oil_obj.imported.densities
print oil_obj.imported.api

<Oil("COOPER BASIN LIGHT NAPHTHA, SANTOS LTD")>
[<Density(660.0 kg/m^3 at 288.15K)>]
None


Here we can see that the original imported record contains a single measured density, but has no API value.<br>
The API can be computed from the measured density.

In this case, the measured density is a 288.15K, which is $ 15^\circ C $, the standard tempuerature for API, so the conversion can be done directly.

There are records in the oil library in which the densities are measured at temperatures quite far from the standard $ 15^\circ C $ <br>

In this case, the density at $ 15^\circ C $ can be estimated from teh measured values using a standard expansion coefficient.

When interpolating a density value at a measured temperature to one at a desired temperature, we use the following formula(s):

$$
\begin{align}
k_{\rho T} &= 0.008 K^{-1} \cr
T_{ref} &= \text{our reference temperature} \cr
T_0 &= \text{our desired temperature} \cr
\rho_{ref} &= \rho_0 \cdot (1 - k_{\rho T} \cdot (T_{ref} - T_0)) \cr
&\text{thus...} \cr
\rho_0 &= {\rho_{ref} \over (1 - k_{\rho T} \cdot (T_{ref} - T_0))} \cr
\end{align}
$$

Let's apply this method to our reference density.

In [16]:
density_at_temp = 660.0 / (1 - 0.008 * (288.0 - 288.15))
print density_at_temp

659.208949261


We now have the density at $ 15^\circ C $, which we will use to estimate the API value.<br>
And as it so happens, we have packaged this formula into a convenience function.

In [27]:
from oil_library.utilities.estimations import density_at_temp
rho = oil_obj.densities[0].kg_m_3
ref_temp = oil_obj.densities[0].ref_temp_k
print rho, ref_temp
density_at_temp(rho, ref_temp, 288.15)

660.0 288.15


660.0

In [ ]:
api = (141.5 * 1000 / density_at_temp) - 131.5
print api

...and as you can see, this is the API value that our oil object contains.

In [ ]:
print oil_obj.api